In [ ]:
import os
import pandas as pd
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts
import holoviews as hv

from inter_view.dashboards import SegmentationDashBoard

hv.extension('bokeh', width=100)

# select the pipeline output folder 

In [ ]:
# set the working path, not needed if pipeline was configured with absolute paths (instead of relative as in luigi.cfg example)
os.chdir('../')

basedir='data/workflow_out_20210304-114647'

In [ ]:
df = pd.read_hdf(os.path.join(basedir, 'output_collection.h5'))
# l100seg_props = pd.read_hdf(os.path.join(basedir, 'l100seg_props.h5'))
# spx_props = pd.read_hdf(os.path.join(basedir, 'spx_props.h5'))
flat_props = pd.read_hdf(os.path.join(basedir, 'flat_props.h5'))
flat_props.reset_index('timepoint', inplace=True)
flat_props = flat_props.join(flat_props.index.to_frame())

if 'CENTER_SURROUND' in df.index.get_level_values('subdir').unique():
    # keep only MIP (zslice=1) or middle plane zslice
    df = df.groupby(['platedir', 'plate_row', 'plate_column'], as_index=False, group_keys=False) \
           .apply(lambda x:x.dc[:,:,:,:,:,x.dc[:, ['FG_MASK', 'CENTER_SURROUND']].index.get_level_values('zslice')])
else:
    # keep only subdir of interest and zslice>0
    df = df.dc[:,['TIF_OVR', 'TIF_OVR_MIP', 'FG_MASK', 'SEGMENTS_L3', 'SHADING_MASK', 'CENTER_SURROUND', 'SEGMENTS_L100', 'SUPER_PX', 'SKELETON'],:,:,:,1].copy()

df.reset_index(['subdir', 'channel', 'zslice'], inplace=True)

# combine subdir and channel in a single index
df['layer'] = df.subdir
ovr_filter = df.subdir.isin(['TIF_OVR_MIP', 'TIF_OVR'])
df.loc[ovr_filter, 'layer'] +=  '_' + df.loc[ovr_filter, 'channel'].astype(str)

# add analysis type index (middle plane or MIP)
df['analysis'] = df.subdir.map({'TIF_OVR':'middle', 'TIF_OVR_MIP':'MIP', 'FG_MASK':'MIP', 'SEGMENTS_L3':'MIP', 'SHADING_MASK':'MIP', 'CENTER_SURROUND':'middle', 'SEGMENTS_L100':'MIP', 'SUPER_PX':'middle', 'SKELETON':'MIP'})

df = df.set_index(['layer', 'analysis'], append=True).sort_index()

In [ ]:
from functools import lru_cache
from skimage.io import imread

channel_config = {'TIF_OVR_MIP_1':{'cmap':'blue', 'intensity_bounds':(300,10000), 'slider_limits':(0,20000)},
                  'TIF_OVR_MIP_2':{'cmap':'green', 'intensity_bounds':(200,3000), 'slider_limits':(0,20000)},
                  'TIF_OVR_MIP_3':{'cmap':'red', 'intensity_bounds':(100,1200), 'slider_limits':(0,20000)},
                  'TIF_OVR_MIP_4':{'cmap':'red', 'intensity_bounds':(100,5000), 'slider_limits':(0,20000)},
                  'TIF_OVR_1':{'cmap':'blue', 'intensity_bounds':(300,10000), 'slider_limits':(0,20000)},
                  'TIF_OVR_2':{'cmap':'green', 'intensity_bounds':(200,3000), 'slider_limits':(0,20000)},
                  'TIF_OVR_3':{'cmap':'red', 'intensity_bounds':(100,1200), 'slider_limits':(0,20000)},
                  'TIF_OVR_4':{'cmap':'red', 'intensity_bounds':(100,5000), 'slider_limits':(0,20000)},
                  'FG_MASK':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.5},
                  'SEGMENTS_L3':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},
                  'SHADING_MASK':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},
                  'CENTER_SURROUND':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},
                  'SEGMENTS_L100':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},
                  'SUPER_PX':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.5},
                  'SKELETON':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'max', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},}

# opts.defaults(opts.Image('channel', frame_width=1000))
opts.defaults(opts.Image('channel.FG_MASK', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.SEGMENTS_L3', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1), tools=['hover']))
opts.defaults(opts.Image('channel.SHADING_MASK', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.CENTER_SURROUND', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.SEGMENTS_L100', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1), tools=['hover']))
opts.defaults(opts.Image('channel.SUPER_PX', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.SKELETON', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))



# custom file reading
# reads only the first slice
# caches the last set of channels 
@lru_cache(maxsize=4)
def read_first_slice(path):
    return imread(path, img_num=0)

seg_db = SegmentationDashBoard(df=df,
                               multi_select_levels=['layer'],
                               channel_config=channel_config,
                               composite_channels=['TIF_OVR_MIP_1', 'TIF_OVR_MIP_2', 'TIF_OVR_MIP_3', 'TIF_OVR_MIP_4', 'TIF_OVR_1', 'TIF_OVR_2', 'TIF_OVR_3', 'TIF_OVR_4'],
                               overlay_channels=['CENTER_SURROUND', 'FG_MASK', 'SEGMENTS_L100', 'SEGMENTS_L3', 'SHADING_MASK', 'SKELETON', 'SUPER_PX'],
                               loading_fun=read_first_slice,
                              )

seg_db.file_widgets[3].value = ['TIF_OVR_MIP_1', 'TIF_OVR_MIP_2', 'TIF_OVR_MIP_3', 'TIF_OVR_MIP_3',
                            'TIF_OVR_1', 'TIF_OVR_2', 'TIF_OVR_3']
seg_db.file_widgets[4].value = 'MIP'

# seg_db.panel().servable()

In [ ]:
from inter_view.dashboards import ScatterDashBoard

opts.defaults(opts.Points('props_scatter', frame_width=600, frame_height=600, fontsize={'legend': 6}, size=5, alpha=0.5, line_color=None,
                          nonselection_fill_alpha=0.1, selection_fill_alpha=0.8, selection_line_color='black', selection_line_alpha=1.,))

scat_db = ScatterDashBoard(df=flat_props.reset_index(drop=True).set_index('timepoint', drop=False).sort_index(axis=1).sample(len(flat_props)),
                           x_key='grid_aspect_ratio',
                           y_key='obj_ch2_mass_displacement',
                           color_key='timepoint',
                           hover_keys=['platedir', 'plate_row', 'plate_column'])

# scat_db.panel().servable()

In [ ]:
import param
import panel as pn
import numpy as np

# wrap in class just of easier callbacks definition
# don't care about global "sub-dashboards"
class DashBoard(param.Parameterized):
    
    seg_db = param.Parameter(seg_db)
    scat_db = param.Parameter(scat_db)
    
    _updating = param.Boolean(False)
    
    @param.depends('scat_db.selected_row', watch=True)
    def _load_selection_image(self):
        '''load image corresponding to first point in selection'''
        if not self._updating:
            self._updating = True
            self.seg_db.loading_off = True
            self.seg_db.file_widgets[0].value = self.scat_db.selected_row.platedir
            self.seg_db.file_widgets[1].value = self.scat_db.selected_row.plate_row
            self.seg_db.file_widgets[2].value = self.scat_db.selected_row.plate_column
            self.seg_db.loading_off = False

            # trigger update once all values have been set
            self.seg_db.param.trigger('subdf')
            self._updating = False
        
    @param.depends('seg_db.subdf', watch=True)
    def _select_image_pt(self):
        '''select scatter point corresponding to opened image'''
        
        if not self._updating:
            self._updating = True
            idx = np.argwhere( (self.scat_db.subdf[['platedir', 'plate_row', 'plate_column']] == self.seg_db.subdf.index[0][:3]).all(axis=1).values)
            if len(idx) > 0:
                scat_db.selection_ids = list(idx[0])
            else:
                scat_db.selection_ids = []
            
            self._updating = False
    
    def panel(self):
        return pn.Row(scat_db.panel(), seg_db.panel()).servable()

db = DashBoard()

db.panel()